In [1]:
import json 

In [2]:
with open("/home/turning/Jainit/TANQ/EVALUATION_OF_MODELS/gemini/wikibio/test.data", "r") as f:
    data = f.readlines()
    

In [3]:
text_to_id = {}
for i, line in enumerate(data):
    text_to_id[i] = line.strip()

In [4]:
len(data)

72731

In [5]:
len(text_to_id)

72731

In [6]:
json.dump(text_to_id, open("text_to_id.json", "w"))

In [33]:
import json
import re
import pandas as pd

def extract_table(raw_text):
    """
    Extracts a table from a raw text string which may be in one of several formats:
    - Plain text with code fences (e.g., starting with ```text and ending with ```)
    - A JSON string that contains a "result" key with a table string, possibly wrapped in triple quotes.
    
    The table rows are expected to be separated by the literal "<NEWLINE>" token,
    and each row is a pipe-separated list of fields.
    """
    text = raw_text.strip()
    text = text.replace("```text", "").replace("```", "").strip()
    text = text.replace("```", "").strip()
    text = text.replace("\"\"\"", "").strip()
    # --- Step 1. If the input is JSON, extract the "result" field ---
    try:
        parsed = json.loads(text)
        if isinstance(parsed, dict) and "result" in parsed:
            text = parsed["result"]
    except json.JSONDecodeError:
        # Not valid JSON, so assume it's a plain string
        pass

    # --- Step 2. Remove markdown code fences if present ---
    # Remove starting fence like ```text (or any language) and the ending ```
    text = re.sub(r'^```(?:[^\n]*)\n', '', text)
    text = re.sub(r'\n```$', '', text)

    # --- Step 3. Remove extra triple quotes if present ---
    text = text.strip()
    if text.startswith('"""') and text.endswith('"""'):
        text = text[3:-3].strip()

    # --- Step 4. Split the string into rows using the <NEWLINE> token ---
    if "<NEWLINE>" in text:
        rows = text.split("<NEWLINE>")
    else:
        rows = text.split("\n")
    records = []
    for row in rows:
        row = row.strip()
        if not row:
            continue
        # Remove any leading/trailing pipes and split on "|"
        # Then filter out any empty strings after stripping whitespace
        parts = [p.strip() for p in row.strip('|').split("|") if p.strip()]
        if len(parts) >= 2:
            key = parts[0]
            # In case there are additional tokens, join them with " | " to preserve inner separators.
            value = " | ".join(parts[1:])
            records.append((key, value))
    
    # --- Step 5. Create a DataFrame from the records ---
    df = pd.DataFrame(records, columns=["Field", "Value"])
    return df

# --- Example 1: Table wrapped in a markdown code fence ---
raw_data2 = """```text\n| name | john \"jack\" reynolds | <NEWLINE> | birth date | 21 february 1869 | <NEWLINE> | death date | 12 march 1917 | <NEWLINE> | clubs | west bromwich albion, aston villa, celtic, southampton | <NEWLINE> | international appearances | ireland (5), england (8) | <NEWLINE> | honours | FA Cup (1892 with West Bromwich Albion), English League titles (3 with Aston Villa), FA Cups (2 with Aston Villa) | <NEWLINE> | death place | sheffield |\n```\n"""
# --- Example 2: JSON string containing the table in the "result" key ---
raw_data1 = """| death place | sheffield, england | <NEWLINE> | name | jack reynolds | <NEWLINE> | death date | 12 march 1917 | <NEWLINE> | birth date | 21 february 1869 | <NEWLINE> | fullname | john reynolds |"""

# --- Parse and display the tables ---
df1 = extract_table(raw_data1)
df2 = extract_table(raw_data2)

print("Table from Example 1:")
print(df1.to_markdown(index=False))
print("\nTable from Example 2:")
print(df2.to_markdown(index=False))


Table from Example 1:
| Field       | Value              |
|:------------|:-------------------|
| death place | sheffield, england |
| name        | jack reynolds      |
| death date  | 12 march 1917      |
| birth date  | 21 february 1869   |
| fullname    | john reynolds      |

Table from Example 2:
| Field                     | Value                                                                                                             |
|:--------------------------|:------------------------------------------------------------------------------------------------------------------|
| name                      | john "jack" reynolds                                                                                              |
| birth date                | 21 february 1869                                                                                                  |
| death date                | 12 march 1917                                                                     

In [34]:
import json 
with open("/home/turning/Jainit/TANQ/EVALUATION_OF_MODELS/gemini/wikibio/top_tables_wiki.json", "r") as f:
    gt_tables = json.load(f)

In [35]:
gt_tables_dict = {}
for tables in gt_tables:
    for k, v in tables.items():
        gt_tables_dict[k] = v

In [36]:
import pickle as pkl 
with open("/home/turning/Jainit/TANQ/EVALUATION_OF_MODELS/gpt/wikibio/wikibio_4o_output.pkl", "rb") as f:
    pred_tables = pkl.load(f)

In [37]:
pred_tables

[{'67': "```| name | mike saunders | <NEWLINE> | nationality | jamaican | <NEWLINE> | occupation | retired soccer player, assistant coach | <NEWLINE> | college team | university of connecticut huskies | <NEWLINE> | professional teams | connecticut wolves, new york fever, wichita wings, carolina dynamo, minnesota thunder | <NEWLINE> | notable achievements | usl a-league championship (1999) | <NEWLINE> | coaching role | assistant coach, university of the district of columbia men's soccer team (2010) |```"},
 {'82': '"""| name | charles john barnett | <NEWLINE> | birth date | 12 march 1790 | <NEWLINE> | death date | 4 august 1856 | <NEWLINE> | occupation | british army officer, diplomat | <NEWLINE> | military rank | captain (1812), lieutenant colonel (1821) | <NEWLINE> | battles | battle of waterloo (1815) | <NEWLINE> | military service | 3rd regiment of foot | <NEWLINE> | diplomatic posts | consul in warsaw (31 may 1833), consul-general in egypt (11 may 1841 - 17 august 1846) | <NEWLINE>

In [38]:
pred_tables_dict = {}
for tables in pred_tables:
    for k, v in tables.items():
        pred_tables_dict[k] = v

In [39]:
input_tables = {}
for k,v in gt_tables_dict.items():
    input_tables[k] = [extract_table(v).to_markdown(index=False), extract_table(pred_tables_dict[k]).to_markdown(index=False)]

In [42]:
print(input_tables['36877'][0])

| Field        | Value                         |
|:-------------|:------------------------------|
| number       | 20                            |
| birth place  | chicago, illinois             |
| career start | 2014                          |
| high school  | fenwick (oak park, illinois)  |
| draft team   | minnesota lynx                |
| nationality  | american                      |
| draft year   | 2014                          |
| league       | wnba                          |
| name         | tricia liston                 |
| draft league | wnba                          |
| team         | minnesota lynx minnesota lynx |
| birth date   | 20 february 1992              |


In [43]:
print(input_tables['36877'][1])

| Field        | Value                                   |
|:-------------|:----------------------------------------|
| name         | patricia maureen "tricia" liston        |
| birth date   | 20 february 1992                        |
| birth place  | chicago, illinois                       |
| occupation   | professional basketball player          |
| high school  | fenwick high school, oak park, illinois |
| drafted year | 2014                                    |
| drafted by   | minnesota lynx                          |


In [44]:
json.dump(input_tables, open("input_tables_for_score.json", "w"))